In [375]:
from datetime import datetime as dt
from IPython.display import display, HTML
import urllib
import pandas as pd

In [376]:
collection_input='conservation-area'
organisation_input=''
severity_input=[] # list of severities e.g ["error", "warning", "info", "notice"]
line_number_input=''

datasette_url = "https://datasette.planning.data.gov.uk/"

current_date = dt.now().date()

date_query = f" where SUBSTRING(s.entry_date, 1, 4) = '2023'"

query = ""
if collection_input:
    query = f" and s.collection = '{collection_input}'"

if organisation_input:
    query = query + f" and s.organisation = '{organisation_input}'"

params = urllib.parse.urlencode({
    "sql": f"""
    select re.resource, re.endpoint, s.organisation, s.entry_date, e.endpoint_url
    from resource_endpoint re
    inner join endpoint e
    on re.endpoint = e.endpoint
    inner join source s
    on e.endpoint = s.endpoint
    {date_query}
    {query}

    """,
    "_size": "max"
})

url = f"{datasette_url}digital-land.csv?{params}"
df = pd.read_csv(url)
if (df.empty):
    print(print("\033[1m No results found"))
else:
    print("\033[1m Resources and endpoints added this year for", collection_input,": ")
df


 Resources and endpoints added this year for conservation-area : 


,resource,endpoint,organisation,entry_date,endpoint_url
0,2c62d64f790d690488b5c523ac1f0b3080848708c767b9...,3359f0dcc3566387183cdf903722f1771dde9cba1f16c7...,local-authority-eng:CMD,2023-08-14T09:09:18Z,https://opendata.camden.gov.uk/api/views/cbz9-...
1,7306c46664b948b9d0ded55f661aa150734829d0c2d924...,3359f0dcc3566387183cdf903722f1771dde9cba1f16c7...,local-authority-eng:CMD,2023-08-14T09:09:18Z,https://opendata.camden.gov.uk/api/views/cbz9-...
2,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,ecf6d0fbf0312df98a2f7e8e8b41a65757a5f1eb3ad318...,local-authority-eng:SWK,2023-08-15T13:13:41Z,https://www.southwark.gov.uk/assets/attach/194...


In [377]:
params = urllib.parse.urlencode({
    "sql": f"""
    select description, issue_type, severity
    from issue_type
    """,
    "_size": "max"
})

url = f"{datasette_url}digital-land.csv?{params}"
issue_type = pd.read_csv(url)

In [378]:
# resources = df['resource'].tolist()
# resources_param_string_starter = "'"
# resources_param_string = resources_param_string_starter + "', '".join(resources) + "'"
# params = urllib.parse.urlencode({
#     "sql": f"""
#     select field,issue_type,dataset,resource,value
#     from issue
#     where resource in ({resources_param_string})
#     """,
#     "_size": "max"
# })

# url = f"{datasette_url}{collection_input}.csv?{params}"
# df1 = pd.read_csv(url)
# print("\033[1m List of issues with conservation area: ")

query=""
if line_number_input:
    query = f" and line_number = '{line_number_input}'"

resources = df['resource'].tolist()
issues = []
for resource in resources:
    params = urllib.parse.urlencode({
    "sql": f"""
    select field,issue_type,dataset,resource,value, line_number
    from issue
    where resource = '{resource}'
    {query}
    """,
    "_size": "max"
    })
    url = f"{datasette_url}{collection_input}.csv?{params}"
    df1 = pd.read_csv(url)
    issues.append(df1)
df1 = pd.concat(issues, ignore_index=True)
issues[0]
df1

print("\033[1m Issues with types: ")
issues_with_type = df1.merge(issue_type, left_on='issue_type', right_on='issue_type')

 Issues with types: 


In [381]:
df_output = df.join(issues_with_type.set_index('resource'), on='resource')
df_output = df_output.reindex(columns=['organisation', 'dataset', 'endpoint_url', 'entry_date', 'field', 'line_number', 'issue_type', 'value', 'severity', 'description']).set_index('organisation')
if (severity_input):
    df_output = df_output.loc[df_output['severity'].isin(severity_input)]

print("\033[1m Final output: ")
display(HTML(df_output.to_html()))


 Final output: 


,dataset,endpoint_url,entry_date,field,line_number,issue_type,value,severity,description
organisation,,,,,,,,,
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,organisation,2,default-value,local-authority-eng:CMD,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,entry-date,2,default-value,2022-10-14,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,organisation,3,default-value,local-authority-eng:CMD,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,entry-date,3,default-value,2022-10-14,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,organisation,4,default-value,local-authority-eng:CMD,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,entry-date,4,default-value,2022-10-14,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,organisation,5,default-value,local-authority-eng:CMD,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,entry-date,5,default-value,2022-10-14,info,The value has been derrived from other data.
local-authority-eng:CMD,conservation-area,https://opendata.camden.gov.uk/api/views/cbz9-k9zc/rows.csv?accessType=DOWNLOAD,2023-08-14T09:09:18Z,organisation,6,default-value,local-authority-eng:CMD,info,The value has been derrived from other data.
